In [0]:
dbutils.widgets.text("data_source","")
v_data_source = dbutils.widgets.get("data_source")

In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common_Functions"

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,DateType

In [0]:
schema = StructType([
    StructField('raceId',IntegerType(),True),
    StructField('year',IntegerType(),True),
    StructField('round',IntegerType(),True),
    StructField('circuitId',IntegerType(),True),
    StructField('name',StringType(),True),
    StructField('date',DateType(),True),
    StructField('time',StringType(),True),
    StructField('url',StringType(),True),
])


df = spark.read.csv("abfss://udayadbcontainerv1@udayadbsav1.dfs.core.windows.net/RAW-V1/races.csv",schema=schema,header=True)
df.display()



In [0]:
from pyspark.sql.functions import col, lit, concat, to_timestamp

In [0]:
df = df.withColumn("data_source",lit(v_data_source))\
       .withColumn("Date_Time", to_timestamp(concat(col('date'),lit(" "), col('time')),"yyyy-MM-dd HH:mm:ss"))

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
df = add_ingestion_date(df)

In [0]:
df.display()

In [0]:
# Renaming Columns 
df = df.withColumnRenamed('raceid','race_id')\
       .withColumnRenamed('circuitid','circuit_id')

In [0]:
df= df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('Date_Time').alias('race_timestamp'))

In [0]:
df.display()

In [0]:
df.write.mode("overwrite").partitionBy('race_year').parquet(f"{Processed_path}/races")

In [0]:
df.display()

In [0]:
dbutils.notebook.exit("Sucessed")